# 5.1 학습 모델을 저장하고 재사용하기

모델을 저장하고 재사용하는 방법을 익혀봅니다.

In [1]:
import tensorflow as tf
import numpy as np

C:\Users\kist1\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
data = np.loadtxt('./data.csv', delimiter=',',
                  unpack=False, dtype='float32')

In [14]:
data

array([[0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0.],
       [1., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1.]], dtype=float32)

In [15]:
data.shape

(6, 5)

In [17]:
data[:,0:2] # 1열, 2열 

array([[0., 0.],
       [1., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.]], dtype=float32)

In [18]:
data[:,2:] # 3열, 4열, 5열

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

unpack : bool, optional

If True, the returned array is transposed, so that arguments may be unpacked using x, y, z = loadtxt(...). When used with a structured data-type, arrays are returned for each field. Default is False.

In [5]:
data = np.loadtxt('./data.csv', delimiter=',',
                  unpack=True, dtype='float32')

털, 날개, 기타, 포유류, 조류

x_data = 0, 1

y_data = 2, 3, 4

In [6]:
data

array([[0., 1., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1.]], dtype=float32)

In [7]:
data.shape

(5, 6)

In [8]:
data[0:2] # 1행과 2행

array([[0., 1., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1.]], dtype=float32)

In [9]:
data[2:] # 3행, 4행, 5행

array([[1., 0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1.]], dtype=float32)

In [10]:
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [11]:
x_data

array([[0., 0.],
       [1., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.]], dtype=float32)

In [12]:
y_data

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

# 신경망 모델 구성

학습에 직접적으로 사용하지 않고 학습 횟수에 따라 단순히 증가시킬 변수를 만듭니다.

In [13]:
global_step = tf.Variable(0, trainable=False, name='global_step')

In [14]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
# global_step로 넘겨준 변수를, 학습용 변수들을 최적화 할 때 마다 학습 횟수를 하나씩 증가시킵니다.
train_op = optimizer.minimize(cost, global_step=global_step)

+ 3개의 레이어 => hidden layer 2개

+ 레이어를 늘리는게 무조건 좋은 것은 아니다. => 과적합(overfitting) 문제

# 신경망 모델 학습

In [16]:
sess = tf.Session()
# 모델을 저장하고 불러오는 API를 초기화합니다.
# global_variables 함수를 통해 앞서 정의하였던 변수들을 저장하거나 불러올 변수들로 설정합니다.
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

+ checkpoint file: 학습된 모델을 저정한 파일

In [17]:
# 최적화 진행
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

# 최적화가 끝난 뒤, 변수를 저장합니다.
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

Step: 1,  Cost: 1.082
Step: 2,  Cost: 1.035


'./model/dnn.ckpt-2'

+ global_step=global_step: 최적화 함수가 학습용 변수들을 최적화 할 때마다 global_step 변수의 값이 1씩 증가

# 결과 확인

0: 기타 1: 포유류, 2: 조류

In [18]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 0 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 83.33


한 번 더 실행

In [19]:
# 최적화 진행
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

# 최적화가 끝난 뒤, 변수를 저장합니다.
saver.save(sess, './model/dnn.ckpt', global_step=global_step)
    
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

Step: 3,  Cost: 0.994
Step: 4,  Cost: 0.962
예측값: [0 0 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 83.33


In [20]:
# 최적화 진행
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

# 최적화가 끝난 뒤, 변수를 저장합니다.
saver.save(sess, './model/dnn.ckpt', global_step=global_step)
    
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

Step: 5,  Cost: 0.931
Step: 6,  Cost: 0.903
예측값: [0 0 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 83.33


In [21]:
# 최적화 진행
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

# 최적화가 끝난 뒤, 변수를 저장합니다.
saver.save(sess, './model/dnn.ckpt', global_step=global_step)
    
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

Step: 7,  Cost: 0.876
Step: 8,  Cost: 0.852
예측값: [0 0 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 83.33


In [22]:
# 최적화 진행
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

# 최적화가 끝난 뒤, 변수를 저장합니다.
saver.save(sess, './model/dnn.ckpt', global_step=global_step)
    
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

Step: 9,  Cost: 0.828
Step: 10,  Cost: 0.807
예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00


# 5.2 텐서보드 사용하기

tf.name_scope를 사용

In [2]:
data = np.loadtxt('./data.csv', delimiter=',',
                  unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

# 신경망 모델 구성

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# with tf.name_scope 으로 묶은 블럭은 텐서보드에서 한 레이어안에 표현해줍니다
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

    # tf.summary.scalar 를 이용해 수집하고 싶은 값들을 지정할 수 있습니다.
    tf.summary.scalar('cost', cost)

+ tf.variable에 name을 지정해주면 해당 이름의 변수가 어디서 사용되었는지 쉽게 알 수 있음

In [3]:
# 신경망 모델 학습

sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

# 텐서보드에서 표시해주기 위한 텐서들을 수집합니다.
merged = tf.summary.merge_all()
# 저장할 그래프와 텐서값들을 저장할 디렉토리를 설정합니다.
writer = tf.summary.FileWriter('./logs', sess.graph)
# 이렇게 저장한 로그는, 학습 후 다음의 명령어를 이용해 웹서버를 실행시킨 뒤
# tensorboard --logdir=./logs
# 다음 주소와 웹브라우저를 이용해 텐서보드에서 확인할 수 있습니다.
# http://localhost:6006

# 최적화 진행
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    # 적절한 시점에 저장할 값들을 수집하고 저장합니다.
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

saver.save(sess, './model/dnn.ckpt', global_step=global_step)

INFO:tensorflow:Restoring parameters from ./model\dnn.ckpt-100
Step: 1,  Cost: 0.550
Step: 2,  Cost: 0.550
Step: 3,  Cost: 0.550
Step: 4,  Cost: 0.550
Step: 5,  Cost: 0.550
Step: 6,  Cost: 0.550
Step: 7,  Cost: 0.550
Step: 8,  Cost: 0.550
Step: 9,  Cost: 0.550
Step: 10,  Cost: 0.550
Step: 11,  Cost: 0.550
Step: 12,  Cost: 0.550
Step: 13,  Cost: 0.550
Step: 14,  Cost: 0.550
Step: 15,  Cost: 0.550
Step: 16,  Cost: 0.550
Step: 17,  Cost: 0.550
Step: 18,  Cost: 0.550
Step: 19,  Cost: 0.550
Step: 20,  Cost: 0.550
Step: 21,  Cost: 0.550
Step: 22,  Cost: 0.550
Step: 23,  Cost: 0.550
Step: 24,  Cost: 0.550
Step: 25,  Cost: 0.550
Step: 26,  Cost: 0.550
Step: 27,  Cost: 0.550
Step: 28,  Cost: 0.550
Step: 29,  Cost: 0.550
Step: 30,  Cost: 0.550
Step: 31,  Cost: 0.550
Step: 32,  Cost: 0.550
Step: 33,  Cost: 0.550
Step: 34,  Cost: 0.550
Step: 35,  Cost: 0.550
Step: 36,  Cost: 0.550
Step: 37,  Cost: 0.550
Step: 38,  Cost: 0.550
Step: 39,  Cost: 0.550
Step: 40,  Cost: 0.550
Step: 41,  Cost: 0.550
Ste

'./model/dnn.ckpt-100'

+ 앞에서 만들었던 모델을 지우고 실행해야 에러가 나지 않음

In [4]:
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
